In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="6IGlv7k8xG4zaAfQhDII")
project = rf.workspace("wenchuan-wu").project("t2_glass")
dataset = project.version(2).download("tensorflow")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to T2_Glass-2 in tensorflow:: 100%|██████████| 529/529 [00:00<00:00, 4781.33it/s]


Save the image paths and their corresponding labels in a CSV file.



In [ ]:
import os
import pandas as pd

# Image data is in a directory called 'images'
image_dir = 'Glasses-and-No-Glasses-1'

# Get all the image paths in the directory
image_paths = []
for dirpath, dirnames, filenames in os.walk(image_dir):
    for filename in filenames:
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_paths.append(os.path.join(dirpath, filename))

# Get the image labels based on the directory names
image_labels = [os.path.basename(os.path.dirname(image_path)) for image_path in image_paths]

# Create a dataframe with the image paths and their corresponding labels
df = pd.DataFrame({'image': image_paths, 'label': image_labels})

# Save the dataframe to a CSV file
df.to_csv('image_data.csv', index=False)

CREATE MODEL

In [ ]:
#Import Library
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load data (dataset is in CSV format with 'image' and 'label' columns)
data = pd.read_csv('image_data.csv')

In [ ]:
# Create image directory
os.makedirs('images', exist_ok=True)

In [ ]:
# Save images from the dataset
for i, row in data.iterrows():
    with open(os.path.join('images', str(i) + '.jpg'), 'wb') as f:
        f.write(row['image'])

# Define image directory, size, and format
image_dir = 'images'
image_size = (150, 150)
image_format = 'jpg'

In [ ]:
# Define the function to load data
def load_data(image_dir, image_size, image_format):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        image_dir,
        target_size=image_size,
        batch_size=32,
        class_mode='categorical')

    test_generator = test_datagen.flow_from_directory(
        image_dir,
        target_size=image_size,
        batch_size=32,
        class_mode='categorical')

    return train_generator, test_generator

In [ ]:
# Load the train and test datasets
train_generator, test_generator = load_data(image_dir, image_size, image_format)

In [ ]:
# Reshape the images
train_images = train_generator.next()[0].reshape(10000, 150, 150, 3)
test_images = test_generator.next()[0].reshape(2500, 150, 150, 3)

# Reshape the labels
train_labels = train_generator.next()[1]
test_labels = test_generator.next()[1]

Calculate Accuracy Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=25, validation_data=(test_images, test_labels))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_images, test_labels)
print('Test accuracy:', accuracy)

To test on virtual glasses, you need to provide a pair of virtual glasses images. The input image will be passed through the model to obtain the predictions.

In [ ]:
def test_image(file_path):
    img = load_img(file_path, target_size=(150, 150))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)

    return np.argmax(prediction)

print(test_image('path_to_image.jpg'))

SAVE MODEL

In [ ]:
from tensorflow.keras.models import load_model

# save the model
model.save('my_model.h5')

# load the saved model
loaded_model = load_model('my_model.h5')

Try on virtual glasses using the camera model

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('model.h5')

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
def preprocess_image(frame):
    # Preprocessing steps
    while True:
    ret, frame = cap.read()
    if not ret:
        break
    preprocessed_frame = preprocess_image(frame)
    prediction = model.predict(preprocessed_frame)
    # Blending and displaying the predicted virtual glasses frame
    return frame

In [ ]:
#Release the camera
cap.release()

In [ ]:
#Close Program
cv2.destroyAllWindows()